**squares** = A1

**unit** = collection of nine squares, being either one full row, column, or box

**peers** = squares that share a unit

## Emilio

In [22]:
import json

def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [a+b for a in A for b in B]

digits   = '123456789'
rows     = 'ABCDEFGHI'
cols     = digits
squares  = cross(rows, cols)
unitlist = ([cross(rows, c) for c in cols] +
            [cross(r, cols) for r in rows] +
            [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')])
units = dict((s, [u for u in unitlist if s in u]) for s in squares)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in squares)
print(peers)

# print([u for u in unitlist])


{'A1': {'A7', 'C3', 'A4', 'B3', 'A5', 'A3', 'C2', 'A9', 'H1', 'C1', 'A6', 'A2', 'D1', 'E1', 'A8', 'F1', 'G1', 'I1', 'B2', 'B1'}, 'A2': {'A1', 'A7', 'C3', 'A4', 'H2', 'B3', 'A5', 'I2', 'A3', 'C2', 'A9', 'C1', 'A6', 'F2', 'D2', 'E2', 'A8', 'G2', 'B2', 'B1'}, 'A3': {'A1', 'A7', 'C3', 'A4', 'G3', 'B3', 'A5', 'C2', 'A9', 'C1', 'A6', 'A2', 'D3', 'E3', 'H3', 'A8', 'I3', 'B2', 'B1', 'F3'}, 'A4': {'A1', 'A7', 'A5', 'E4', 'A3', 'C4', 'A9', 'B6', 'C6', 'D4', 'G4', 'H4', 'A6', 'B5', 'I4', 'A8', 'C5', 'F4', 'B4', 'A2'}, 'A5': {'A1', 'A7', 'A4', 'A3', 'C4', 'A9', 'B6', 'I5', 'C6', 'F5', 'A6', 'D5', 'E5', 'B5', 'G5', 'A8', 'H5', 'C5', 'B4', 'A2'}, 'A6': {'A1', 'A7', 'A4', 'A5', 'A3', 'C4', 'A9', 'B6', 'C6', 'F6', 'G6', 'E6', 'B5', 'I6', 'A8', 'C5', 'H6', 'D6', 'B4', 'A2'}, 'A7': {'A1', 'A4', 'A5', 'C9', 'A3', 'F7', 'A9', 'B7', 'E7', 'C7', 'H7', 'A6', 'I7', 'B8', 'A8', 'C8', 'G7', 'A2', 'D7', 'B9'}, 'A8': {'F8', 'A1', 'D8', 'I8', 'A4', 'A7', 'A5', 'C9', 'A3', 'H8', 'A9', 'B7', 'G8', 'C7', 'A6', 'B8', 

## Roberto

In [23]:
def test():
    "A set of unit tests."
    assert len(squares) == 81
    assert len(unitlist) == 27
    assert all(len(units[s]) == 3 for s in squares)
    assert all(len(peers[s]) == 20 for s in squares)
    assert units['C2'] == [['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2'],
                           ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'],
                           ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']]
    assert peers['C2'] == set(['A2', 'B2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2',
                               'C1', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9',
                               'A1', 'A3', 'B1', 'B3'])
    print('All tests pass.')

## Emilio

In [24]:
def parse_grid(grid):
    """Convert grid to a dict of possible values, {square: digits}, or
    return False if a contradiction is detected."""
    ## To start, every square can be any digit; then assign values from the grid.
    values = dict((s, digits) for s in squares)
    for s,d in grid_values(grid).items():
        if d in digits and not assign(values, s, d):
            return False ## (Fail if we can't assign d to square s.)
    return values

## Roberto

In [25]:
def grid_values(grid):
    "Convert grid into a dict of {square: char} with '0' or '.' for empties."
    chars = [c for c in grid if c in digits or c in '0.']
    assert len(chars) == 81
    return dict(zip(squares, chars))

# CONSTRAINT PROPAGATION
## Emilio

In [26]:
def assign(values, s, d):
    """Eliminate all the other values (except d) from values[s] and propagate.
    Return values, except return False if a contradiction is detected."""
    other_values = values[s].replace(d, '')
    if all(eliminate(values, s, d2) for d2 in other_values):
        return values
    else:
        return False

## Roberto

In [27]:
def eliminate(values, s, d):
    """Eliminate d from values[s]; propagate when values or places <= 2.
    Return values, except return False if a contradiction is detected."""
    if d not in values[s]:
        return values ## Already eliminated
    values[s] = values[s].replace(d,'')
    ## (1) If a square s is reduced to one value d2, then eliminate d2 from the peers.
    if len(values[s]) == 0:
        return False ## Contradiction: removed last value
    elif len(values[s]) == 1:
        d2 = values[s]
        if not all(eliminate(values, s2, d2) for s2 in peers[s]):
            return False
    ## (2) If a unit u is reduced to only one place for a value d, then put it there.
    for u in units[s]:
        dplaces = [s for s in u if d in values[s]]
        if len(dplaces) == 0:
            return False ## Contradiction: no place for this value
        elif len(dplaces) == 1:
            # d can only be in one place in unit; assign it there
                if not assign(values, dplaces[0], d):
                    return False
    return values

## Emilio

In [28]:
def display(values):
    "Display these values as a 2-D grid."NB
    width = 1+max(len(values[s]) for s in squares)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else ''))
                      for c in cols)
        if r in 'CF': print(line)
    print()

SyntaxError: invalid syntax (805916166.py, line 2)

# SEARCH AND SOLVE
## Roberto

In [ ]:
def solve(grid): return search(parse_grid(grid))

## Emilio

In [ ]:
def search(values):
    "Using depth-first search and propagation, try all possible values."
    if values is False:
        return False ## Failed earlier
    if all(len(values[s]) == 1 for s in squares): 
        return values ## Solved!
    ## Chose the unfilled square s with the fewest possibilities
    n,s = min((len(values[s]), s) for s in squares if len(values[s]) > 1)
    return some(search(assign(values.copy(), s, d)) 
		for d in values[s])

## Roberto

In [ ]:
def some(seq):
    "Return some element of seq that is true."
    for e in seq:
        if e: return e
    return False